In [18]:
import os
import random
from shutil import copyfile
random.seed(123)
import numpy as np
import math
from unetModel import *

from skimage import io as img_io 

from keras.preprocessing.image import ImageDataGenerator

In [19]:
HOMEDIR = os.getcwd()
BBBCMASK_DIR = '/BBBC_data/mask_creation_images'
# get number of images
ALL_IMAGES = [f for f in os.listdir(HOMEDIR + BBBCMASK_DIR + '/image') if f.endswith('.TIF')]
N_IMAGES = len(ALL_IMAGES)


In [8]:
# function to split into train and validate sets...
p = 0.1
images_s = random.shuffle(ALL_IMAGES)
ntrain =N_IMAGES-math.floor(p*N_IMAGES)
train = random.sample(ALL_IMAGES,ntrain)
valid = [f for f in ALL_IMAGES if f not in train]

if not os.path.exists(HOMEDIR + BBBCMASK_DIR + '/train'):
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/image')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/label')

    
if not os.path.exists(HOMEDIR + BBBCMASK_DIR + '/valid'):
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/image')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/label')


for f in train:
    raw_image_src = HOMEDIR + BBBCMASK_DIR + '/image/' + f
    raw_image_dest = HOMEDIR + BBBCMASK_DIR + '/train/image/' + f
    mask_image_src = HOMEDIR + BBBCMASK_DIR + '/label/' + f
    mask_image_dest = HOMEDIR + BBBCMASK_DIR + '/train/label/' + f
    copyfile(raw_image_src,raw_image_dest)
    copyfile(mask_image_src,mask_image_dest)

for f in valid:
    raw_image_src = HOMEDIR + BBBCMASK_DIR + '/image/' + f
    raw_image_dest = HOMEDIR + BBBCMASK_DIR + '/valid/image/' + f
    mask_image_src = HOMEDIR + BBBCMASK_DIR + '/label/' + f
    mask_image_dest = HOMEDIR + BBBCMASK_DIR + '/valid/label/' + f
    copyfile(raw_image_src,raw_image_dest)
    copyfile(mask_image_src,mask_image_dest)


    


In [26]:
train_image_dir = HOMEDIR + BBBCMASK_DIR + '/train/image/'
train_mask_dir = HOMEDIR + BBBCMASK_DIR + '/train/label/'


In [23]:
np.max(test_img)

255

In [28]:
# Create data generator

data_gen_args = dict(rotation_range = 90,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

def trainGenerator(batch_size,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        mask_folder,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)



In [29]:
mygen = trainGenerator(20, train_image_dir, train_mask_dir, data_gen_args)